## Efficient Simulation in the Inventory Model


#### John Stachurski

Simulation can be time consuming, especially if we need to simulate many paths

And it's often true that large sample sizes are necessary to get an accurate picture

Let's look at speeding up our simulations using [Numba](https://numba.pydata.org/)

In [1]:
import numpy as np
from numba import jit, prange, vectorize
from quantecon.util import tic, toc

Parameters:

In [2]:
s, S = 10.0, 100.0
p = 0.4
initial_x = 50.0

### Simulating One Path

Let's look at simulating one path without any optimization.

In [3]:
def sim_one_inventory_path_simple(sim_length=1_000_000):
        
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

How fast does this run?

In [4]:
tic()
X = sim_one_inventory_path_simple()
toc()

TOC: Elapsed: 0:00:0.72


0.7226040363311768

Now let's build a jitted version.

In [5]:
@jit(nopython=True)
def sim_one_inventory_path_jitted(sim_length=1_000_000):
    
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length, dtype=np.int32)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

This is **much** faster:

In [6]:
tic()
X = sim_one_inventory_path_jitted()
toc()

TOC: Elapsed: 0:00:0.21


0.2180788516998291

In [7]:
tic()
X = sim_one_inventory_path_jitted()
toc()

TOC: Elapsed: 0:00:0.04


0.04297757148742676

### Simulating Many Paths

Here's a jitted (and hence fast) piece of code for simulating many paths.

In [8]:
@jit(nopython=True)
def sim_paths_jitted(num_paths=500_000,
                     sim_length=1000):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        for m in range(num_paths):
            x, d = X[m, t], dvals[m]
            if x <= s:
                y = max(S - d, 0)
            else:
                y = max(x - d, 0)
            X[m, t+1] = y
    return X


Let's check it's doing what we think it should.

In [9]:
X = sim_paths_jitted(num_paths=6, sim_length=4)

In [10]:
X

array([[50, 49, 47, 45],
       [50, 42, 38, 38],
       [50, 48, 46, 44],
       [50, 49, 49, 49],
       [50, 49, 48, 47],
       [50, 48, 48, 48]], dtype=int32)

Now let's get a reading on the time.

In [11]:
tic()
X = sim_paths_jitted()
toc()

TOC: Elapsed: 0:00:27.70


27.70879316329956

We can do something similar using Numba's `@vectorize` decorator.

In [12]:
@vectorize
def G_vectorized(x, d):
    if x <= s:
        y = max(S - d, 0)
    else:
        y = max(x - d, 0)
    return y


In [13]:
def sim_paths_vectorized(num_paths=500_000,
                         sim_length=1000):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x

    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        X[:, t+1] = G_vectorized(X[:, t], dvals)
    return X

In [14]:
X = sim_paths_vectorized(num_paths=6, sim_length=4)

In [15]:
X

array([[50, 50, 45, 44],
       [50, 47, 47, 46],
       [50, 48, 45, 42],
       [50, 49, 49, 49],
       [50, 50, 49, 49],
       [50, 48, 48, 42]], dtype=int32)

Here's the timing, which is similar to the jitted version.

In [16]:
tic()
X = sim_paths_vectorized()
toc()

TOC: Elapsed: 0:00:25.46


25.46913719177246

We can also target the vectorized function for parallel execution.

In [17]:
@vectorize('float64(float64, float64)', target='parallel')
def G_vectorized_parallel(x, d):
    if x <= s:
        y = max(S - d, 0)
    else:
        y = max(x - d, 0)
    return y

In [18]:
def sim_paths_vectorized_parallel(num_paths=500_000,
                                  sim_length=1000):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        X[:, t+1] = G_vectorized_parallel(X[:, t], dvals)
    return X

In [19]:
X = sim_paths_vectorized_parallel(num_paths=6, sim_length=4)

In [20]:
X

array([[50, 47, 46, 44],
       [50, 47, 45, 45],
       [50, 46, 46, 46],
       [50, 49, 48, 47],
       [50, 49, 49, 49],
       [50, 46, 44, 42]], dtype=int32)

The timing is not much improved because the individual tasks are relatively small, and hence multiple threads are not efficiently exploited.

In [21]:
tic()
X = sim_paths_vectorized_parallel()
toc()

TOC: Elapsed: 0:00:33.27


33.273865699768066

Let's try for more efficient parallelization, where the work is divided into a smaller number of chunks.

In [22]:
@jit(nopython=True, parallel=True)
def sim_paths_jitted_parallel(num_paths=500_000, 
                              sim_length=1000,
                              sections=500):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)

    section_size = num_paths // sections
    
    for i in prange(sections):
        j = i * section_size
        X[j:j+section_size, :] = sim_paths_jitted(num_paths=section_size, 
                                                sim_length=sim_length)
    return X

In [23]:
X = sim_paths_jitted_parallel(num_paths=8, 
                              sim_length=5, 
                              sections=2)

In [24]:
X

array([[50, 50, 50, 48, 48],
       [50, 50, 46, 46, 45],
       [50, 49, 46, 45, 45],
       [50, 50, 50, 48, 47],
       [50, 49, 49, 48, 48],
       [50, 49, 48, 48, 45],
       [50, 50, 50, 43, 43],
       [50, 50, 41, 40, 40]], dtype=int32)

Now the speed up from parallelization is significant.

In [25]:
tic()
X = sim_paths_jitted_parallel()
toc()

TOC: Elapsed: 0:00:2.86


2.8651044368743896